In [18]:
import numpy as np

class Square1:
    def __init__(self):
        self.u = [2, 1, 2, 1, 2, 1, 2, 1]
        self.d = [1, 2, 1, 2, 1, 2, 1, 2]
        self.s = False  # Estado del corte

    def cw(self, layer):
        return [layer[-1]] + layer[:-1]  # Corrige el giro horario

    def ccw(self, layer):
        return layer[1:] + [layer[0]]  # Corrige el giro antihorario

    def slice(self):
        for u in range(6):
            if np.sum(np.array(self.u[:u+1])) == 6:
                number_u = u
                break
        for d in range(6):
            if np.sum(np.array(self.d[:d+1])) == 6:
                number_d = d
                break
        self.u, self.d = self.u[:number_u + 1] + self.d[:number_d+1], self.u[number_u + 1:] + self.d[number_d + 1:]
        self.s = not self.s
        
    def scramble(self, scram):
        scram = scram.strip()
        end_slice = False
        if scram.endswith("/"):
            end_slice = True
            scram = scram[:-1]
        
        # Separar por "/" y limpiar espacios extra
        proc_scram = [m.strip() for m in scram.split("/") if m.strip()]
        
        for movimiento in proc_scram:
            # Eliminar paréntesis y limpiar espacios antes de separar por coma
            u, d = map(int, movimiento.replace("(", "").replace(")", "").split(","))
            self.move(u, d)
            self.slice()
            
        if not end_slice:
            self.slice()

    def move(self, u, d):

        # Capa Inferior (d) - CORREGIDO self.u por self.d
        for i in range(len(self.d) + 1): # +1 para cubrir todos los casos
            possible = False
            if d > 0:
                if np.sum(np.array(self.d[-i:])) == d:
                    possible = True
                    number_d = i # Simplificado: i es el número de piezas
                    break
            elif d < 0:
                if np.sum(np.array(self.d[:i])) == -d:
                    possible = True
                    number_d = i
                    break
            elif d == 0:
                possible = True
                number_d = 0
                break
        
        if not possible:
            raise ValueError(f"Movimiento d={d} no permitido en esta forma")

        # Ejecutar giros d
        if d > 0:
            for _ in range(number_d): self.d = self.cw(self.d)
        elif d < 0:
            for _ in range(number_d): self.d = self.ccw(self.d)
            
    def move(self, u, d):
        
        # comprobamos que se puede hacer el giro
        if u not in range(-5,7):
            raise ValueError("Movimiento no permitido")
        
        if d not in range(-5,7):
            raise ValueError("Movimiento no permitido")
        
        # Giros en la capa superior
        for i in range(len(self.u)):
            possible = False
            if u > 0:
                if np.sum(np.array(self.u[-i:])) == u:
                    possible = True
                    number_u = len(self.u[-i:])
                    break
            elif u < 0:
                if np.sum(np.array(self.u[:i+1])) == -u:
                    possible = True
                    number_u = len(self.u[:i+1])
                    break
            elif u == 0:
                possible = True
                number_u = i
                break
            
        if not possible:
            raise ValueError("Movimiento no permitido")
        
        if u > 0:
            for i in range(number_u):
                self.u = self.cw(self.u)
        elif u < 0:
            for i in range(number_u):
                self.u = self.ccw(self.u)


        # Giros en la capa inferior
        for i in range(len(self.d)):
            possible = False
            if d > 0:
                if np.sum(np.array(self.d[-i:])) == d:
                    possible = True
                    number_d = len(self.u[-i:])
                    break
            elif d < 0:
                if np.sum(np.array(self.d[:i+1])) == -d:
                    possible = True
                    number_d = len(self.d[:i+1])
                    break
            elif d == 0:
                possible = True
                number_d = i
                break
            
        if not possible:
            raise ValueError("Movimiento no permitido")
        
        if d > 0:
            for i in range(number_d):
                self.d = self.cw(self.d)
        elif d < 0:
            for i in range(number_d):
                self.d = self.ccw(self.d)

    def get_shape(self, layer):
        if layer == "u":
            lay = self.u.copy()
        elif layer == "d":
            lay = self.d.copy()
        permutations = [tuple(lay[i+1:]+lay[:i+1]) for i in range(len(lay))]
        dict = {
        (2, 1, 2, 1, 2, 1, 2, 1): "Square",
        (2, 1, 2, 1, 1, 2, 1, 2): "Kite",
        (1, 2, 2, 1, 1, 2, 2, 1): "Barrel",
        (1, 2, 2, 1, 2, 1, 2, 1): "Left Fist",
        (1, 2, 2, 1, 1, 2, 1, 2): "Right Fist",
        (1, 1, 1, 1, 1, 1, 1, 1, 2, 2): "8",
        (1, 1, 1, 1, 1, 1, 2, 2, 2): "6",
        (2, 2, 2, 2, 2, 2): "Star",
        (1, 1, 2, 2, 2, 2, 2): "2",
        (1, 1, 2, 2, 1, 1, 1, 1, 2): "Left 4-2",
        (2, 1, 1, 2, 1, 1, 1, 1, 2): "Right 4-2",
        (2, 1, 1, 1, 1, 1, 2, 1, 2): "Left 5-1",
        (2, 1, 2, 1, 1, 1, 1, 1, 2): "Right 5-1",
        (1, 1, 1, 1, 2, 2, 2, 2): "Scallop",
        (1, 1, 1, 1, 1, 1, 2, 1, 1, 2): "6-2",
        (1, 1, 1, 1, 2, 1, 1, 1, 1, 2): "4-4",
        (2, 1, 1, 2, 1, 1, 1, 2, 1): "3-1-2",
        (2, 1, 1, 1, 2, 1, 1, 2, 1): "3-2-1",
        (1, 2, 1, 2, 2, 2, 2): "L",
        (1, 1, 1, 1, 2, 1, 2, 1, 2): "4-1-1",
        (1, 1, 1, 1, 1, 2, 1, 1, 1, 2): "5-3",
        (2, 2, 2, 1, 2, 1, 1, 1): "Right Pawn",
        (1, 1, 1, 2, 2, 1, 2, 2): "Mushroom",
        (1, 1, 1, 1, 1, 1, 1, 2, 1, 2): "7-1",
        (2, 2, 2, 1, 1, 1, 2, 1): "Left Pawn",
        (1, 1, 2, 1, 1, 2, 1, 1, 2): "2-2-2",
        (2, 1, 1, 2, 2, 2, 1, 1): "Shield",
        (1, 1, 1, 2, 1, 1, 1, 2, 2): "3-3",
        (2, 2, 2, 1, 2, 2, 1): "Bar",
        }
        for perm in permutations:
            if perm in dict:
                return dict[perm]
        
    def get_cubeshape(self, same_order = True):
        a = self.get_shape("u")
        b = self.get_shape("d")
        if same_order:
            ha, hb = hash(a), hash(b)
            if ha < hb:
                return a + " / " + b
            else:
                return b + " / " + a
        return a + " / " + b

In [19]:
import json
import openpyxl
from openpyxl import Workbook

# Cargar archivo JSON
with open("cstimer.json") as f:
    data = json.load(f)

# Extraer la sesión específica
sesion = data['session19']

print(sesion)

[[[0, 25930], ' (0,-4)/ (-5,1)/ (5,-1)/ (-3,0)/ (0,-3)/ (-5,-2)/ (-1,0)/ (3,0)/ (-3,-3)/ (-3,0)/ (0,-1)/ (6,0)', 'revisar csp. me da par', 1723065620], [[0, 12790], ' (-5,-3)/ (3,6)/ (2,-4)/ (3,0)/ (3,0)/ (0,-3)/ (4,0)/ (0,-3)/ (-1,0)/ (1,0)/ (2,0)/ (0,-2)/ (-2,0)/ (-3,0)', '', 1723065682], [[0, 11290], ' (-2,-3)/ (3,0)/ (0,-3)/ (-4,-1)/ (-3,0)/ (0,-3)/ (-2,0)/ (-3,-3)/ (5,0)/ (2,0)/ (4,-2)/ (0,-4)/ (-2,0)', '', 1723065753], [[0, 11670], ' (0,2)/ (4,1)/ (2,-1)/ (4,-5)/ (2,-1)/ (6,-2)/ (-3,0)/ (-2,-3)/ (4,-3)/ (-4,-4)/ (6,0)', '', 1723065847], [[0, 12920], ' (0,-4)/ (0,-3)/ (-5,4)/ (-3,0)/ (0,-3)/ (2,-4)/ (4,0)/ (3,0)/ (3,-3)/ (-5,0)/ (-4,0)/ (-2,-2)/ (-3,0)', '', 1723065896], [[0, 14510], ' (3,5)/ (-5,4)/ (5,-1)/ (4,-2)/ (6,0)/ (-1,0)/ (0,-3)/ (-2,-3)/ (-1,0)/ (4,-2)/ (-4,-1)', '', 1723066038], [[0, 12830], ' (0,5)/ (-2,-2)/ (6,3)/ (-4,-1)/ (-3,0)/ (1,-3)/ (-3,0)/ (5,-2)/ (0,-4)/ (6,-2)/ (4,-2)/ (6,-3)', '', 1723066109], [[0, 17330], ' (4,0)/ (-1,2)/ (0,-3)/ (0,-3)/ (6,0)/ (4,-2)/ (3,-

In [20]:
# import the following csv: csTimerExport_20260102_011237.csv
import pandas as pd
df = pd.read_csv("csTimerExport_20260102_011237.csv", sep=";")
print(df.head())

   No.   Time Comment                                           Scramble  \
0    1  15.46     NaN   (4,0)/ (-3,0)/ (-1,5)/ (3,-3)/ (3,0)/ (6,0)/ ...   
1    2  12.37     NaN   (0,5)/ (-2,-5)/ (-1,2)/ (3,0)/ (-2,-2)/ (3,-1...   
2    3  20.63     NaN   (3,-1)/ (0,-3)/ (0,3)/ (3,0)/ (4,-2)/ (3,0)/ ...   
3    4  11.42     CSP   (0,2)/ (3,0)/ (-5,4)/ (5,-1)/ (3,0)/ (0,-3)/ ...   
4    5  17.97     NaN   (0,-4)/ (4,4)/ (0,-3)/ (3,0)/ (5,-1)/ (-2,0)/...   

                  Date    P.1  
0  2026-01-01 14:36:18  15.46  
1  2026-01-01 14:36:47  12.37  
2  2026-01-01 14:37:54  20.63  
3  2026-01-01 14:38:52  11.42  
4  2026-01-01 14:39:34  17.97  


# Si importamos la sesión en un csv

In [24]:

# Crear un nuevo libro de Excel
wb = Workbook()
sheet = wb.active
sheet.title = "Resultados"

# Agregar encabezados
sheet.append(["Time)", "Scramble", "Parity", "Shape", "Shape (same order)"])

# Procesar los datos de la sesión
def procesar_sesion(df):
    for index, row in df.iterrows():
        tiempo = row['Time']
        mezcla = row['Scramble']
        comentario = row['Comment'] 
        csp = "CSP" if comentario=="CSP" else ""  
        cubo = Square1()
        cubo.scramble(mezcla)
        forma = cubo.get_cubeshape(same_order = False)
        forma_same_order = cubo.get_cubeshape()
  
        sheet.append([tiempo, mezcla, csp, forma, forma_same_order])

# Ejecutar el procesamiento
procesar_sesion(df)

# Guardar el archivo XLSX
wb.save("resultados.xlsx")
print("Archivo 'resultados.xlsx' creado con éxito.")



Archivo 'resultados.xlsx' creado con éxito.


In [ ]:
# import resultados.xlsx and read the content
import openpyxl
wb = openpyxl.load_workbook("resultados.xlsx")
sheet = wb.active

#calcula cuantas veces aparece cada forma
from collections import Counter
shapes = [row[3] for row in sheet.iter_rows(min_row=2, values_only=True)]
shape_counts = Counter(shapes)
# print them but in order
for shape, count in shape_counts.most_common():
    print(f"{shape}: {count}")

# save output in a txt file called shape_counts.txt
with open("shape_counts.txt", "w") as f:
    for shape, count in shape_counts.most_common():
        f.write(f"{shape}: {count}\n")

total_shapes = 30  # total possible shapes
appeared_shapes = len(shape_counts)
print(f"Shapes appeared: {appeared_shapes} out of {total_shapes}")

2 / 4-1-1: 13
2 / 3-1-2: 11
Scallop / Shield: 10
Shield / Right Fist: 9
Left Fist / Shield: 8
4-1-1 / 2: 8
2 / Right 4-2: 8
Left 4-2 / L: 7
2 / 3-3: 7
Shield / Barrel: 7
Kite / Shield: 6
L / Left 5-1: 6
Shield / Left Fist: 6
Barrel / Scallop: 6
Right Pawn / Shield: 6
Scallop / Right Fist: 6
2 / Left 4-2: 6
3-3 / 2: 6
Left 4-2 / 2: 6
Left Pawn / Right Fist: 5
Left Fist / Barrel: 5
Scallop / Barrel: 5
2 / Left 5-1: 5
Shield / Right Pawn: 5
2 / Right 5-1: 5
Right Fist / Left Pawn: 5
2 / 3-2-1: 5
Right Fist / Barrel: 5
L / Left 4-2: 5
Right 4-2 / 2: 5
3-1-2 / 2: 5
2 / 6: 5
4-1-1 / L: 4
Left Fist / Mushroom: 4
Mushroom / Shield: 4
Left Fist / Scallop: 4
Scallop / Mushroom: 4
Mushroom / Left Fist: 4
Barrel / Right Fist: 4
2-2-2 / L: 4
Kite / Right Fist: 4
Shield / Shield: 4
L / 3-2-1: 4
6 / 2: 4
3-3 / L: 4
Left 5-1 / 2: 4
Scallop / Left Fist: 4
L / 3-3: 4
Bar / Left 4-2: 4
Scallop / Left Pawn: 4
Right 5-1 / L: 4
Scallop / Kite: 4
Left Pawn / Mushroom: 4
Shield / Left Pawn: 4
Left Fist / Left

In [35]:
# sacame los mejores tiempos y la mezcla
best_times = {}
for row in sheet.iter_rows(min_row=2, values_only=True):
    tiempo = row[0]
    mezcla = row[1]
    forma = row[3]
    if forma not in best_times or tiempo < best_times[forma][0]:
        best_times[forma] = (tiempo, mezcla)

# print best times
for forma, (tiempo, mezcla) in best_times.items():
    print(f"{forma}: {tiempo} - {mezcla}")

# export best times to best_times.txt
with open("best_times.txt", "w") as f:
    for forma, (tiempo, mezcla) in best_times.items():
        f.write(f"{forma}: {tiempo} - {mezcla}\n")

Left Pawn / Right Fist: 10.69 -  (-2,-3)/ (5,5)/ (0,-3)/ (0,-3)/ (-2,-2)/ (-4,0)/ (6,-3)/ (2,0)/ (-5,0)/
Scallop / Right Pawn: 12.37 -  (0,5)/ (-2,-5)/ (-1,2)/ (3,0)/ (-2,-2)/ (3,-1)/ (6,-3)/ (1,-3)/ (-1,0)/ (4,0)/ (-4,-1)/ (2,0)
Left 4-2 / L: 14.56 -  (0,-4)/ (-3,-3)/ (3,0)/ (0,-3)/ (-2,-5)/ (-3,0)/ (2,0)/ (-3,0)/ (0,-3)/ (2,0)/ (1,-4)/ (-5,0)
Square / Mushroom: 11.42 -  (0,2)/ (3,0)/ (-5,4)/ (5,-1)/ (3,0)/ (0,-3)/ (-2,-3)/ (0,-3)/ (3,0)/ (6,0)/ (0,-3)/ (0,-1)/
2 / 4-1-1: 12.41 -  (0,2)/ (4,-2)/ (5,-4)/ (4,-2)/ (0,-3)/ (6,-1)/ (0,-3)/ (-1,0)/ (1,-4)/ (-4,-4)/ (0,-3)
Left Fist / Shield: 12.10 -  (0,2)/ (0,-3)/ (1,1)/ (0,-3)/ (-3,0)/ (2,-1)/ (0,-5)/ (-3,0)/ (0,-1)/ (0,-5)/ (-3,-4)/ (-1,0)/ (3,-2)/ (3,0)
4-1-1 / L: 11.81 -  (-3,2)/ (4,-2)/ (2,-1)/ (4,-5)/ (-3,-3)/ (5,0)/ (-3,0)/ (0,-1)/ (-4,-5)/ (0,-1)/ (6,0)/ (-1,-2)
Left Fist / Mushroom: 12.67 -  (-3,-1)/ (6,-3)/ (0,-3)/ (-5,-5)/ (3,0)/ (-3,-4)/ (-3,0)/ (-1,0)/ (-4,-5)/ (-1,0)/ (-4,-3)/ (2,0)/
Bar / 4-1-1: 14.63 -  (-2,-3)/ (5,-4)/ (0,

# Si importamos todas las sesiones

In [ ]:
import json
import openpyxl
from openpyxl import Workbook

# Cargar archivo JSON
with open("cstimer.json") as f:
    data = json.load(f)

# Extraer la sesión específica
sesion = data['session19']

print(sesion)

# Crear un nuevo libro de Excel
wb = Workbook()
sheet = wb.active
sheet.title = "Resultados"

# Agregar encabezados
sheet.append(["Time)", "Scramble", "Parity", "Shape", "Shape (same order)"])

# Procesar los datos de la sesión
def procesar_sesion(sesion):
    for item in sesion:
        tiempo = item[0][1] / 1000  # Convertir tiempo a segundos
        mezcla = item[1][1:]  # Mezcla del cubo
        comentario = item[2] if len(item) > 2 else ""  # Comentario si existe
        paridad = "Paridad" if comentario else ""  # Agregar "Paridad" si hay comentario
        cubo = Square1()
        cubo.scramble(mezcla)
        forma = cubo.get_cubeshape(same_order = False)
        forma_same_order = cubo.get_cubeshape()
        # Agregar datos a la hoja de Excel
        sheet.append([tiempo, mezcla, paridad, forma, forma_same_order])

# Ejecutar el procesamiento
procesar_sesion(sesion)

# Guardar el archivo XLSX
wb.save("resultados.xlsx")
print("Archivo 'resultados.xlsx' creado con éxito.")

Archivo 'resultados.xlsx' creado con éxito.
